<a href="https://colab.research.google.com/github/EmmaCOo/Assignment-1.1-Data-Acquisition-with-APIs-and-Scraping/blob/main/Text_Mining_API_and_Scrape_EmmaOo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**ADS 509_ Assignment 1.1: Data Acquisition with APIs and Scraping**

###**Emma Oo**
###09/08/2022



Acquiring data is one of the fundamental steps in any analysis and proficiency at APIs and web scraping unlocks rich data sets for future analyses. In this assignment you will pull data from the Twitter API and scrape lyrics from AZlyrics.com.


Instructions:

1. Create a repository under your GitHub account from this template. Instructions can be found here. 
NOTE: Make your repository public or add your instructor’s GitHub account as a collaborator.

2. Choose two musical artists for your project. Both should have at least 100,000 followers on Twitter and at least twenty songs at their artist page on AZLyrics. Those pages have forms like this.

3. The “API and Scrape.ipynb” file within the repository holds your starting code for the assignment.

   a.  In the first part of the assignment, you will pull data from Twitter. Following the provided code and the instructions in Chapter 2 of your textbook, **pull the descriptions of all followers** for each of your artists.

   b.  The second part of the file holds starting code for the second half of the assignment. Follow the instructions in that notebook, **pulling and storing the lyrics for both artists.**

   c. The final part of the notebook holds evaluation code for the first two parts. If you’ve completed the assignment correctly, you should be able to just run this section as is. 





In [1]:
#install tweepy higher than 4.0
!pip install git+https://github.com/tweepy/tweepy.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/tweepy/tweepy.git to /tmp/pip-req-build-5hl9lt05
  Running command git clone -q https://github.com/tweepy/tweepy.git /tmp/pip-req-build-5hl9lt05


In [2]:
!pip show tweepy

Name: tweepy
Version: 4.10.1
Summary: Twitter library for Python
Home-page: https://www.tweepy.org/
Author: Joshua Roesslein
Author-email: tweepy@googlegroups.com
License: MIT
Location: /usr/local/lib/python3.7/dist-packages
Requires: requests, oauthlib, requests-oauthlib
Required-by: 


##**Twitter API Pull**
###**ARTISTS:  ECOSMITH & SHAYNEWARD**


In [3]:
# for the twitter section
import shutil
import pandas as pd
import tweepy
import os
import datetime
import re
from pprint import pprint

# for the lyrics scrape section
import requests
import time
from bs4 import BeautifulSoup
from collections import defaultdict, Counter

###**Twitter Keys & Tokens**

In [4]:
api_key = 'api-key' 
api_secret = 'api_secret_key'
bearer_token = 'bearer_token'

In [5]:
auth = tweepy.OAuthHandler(api_key, api_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)


In [6]:
client = tweepy.Client(bearer_token, wait_on_rate_limit=True)

###**Testing API**

The Twitter APIs are quite rich. Let's play around with some of the features before we dive into this section of the assignment. For our testing, it's convenient to have a small data set to play with. We will seed the code with the handle of John Chandler, one of the instructors in this course. His handle is @37chandler. Feel free to use a different handle if you would like to look at someone else's data.

We will write code to explore a few aspects of the API:

Pull some of the followers @37chandler.

Explore response data, which gives us information about Twitter users.

Pull the last few tweets by @37chandler.

In [7]:
handle = "37chandler"
user_obj = client.get_user(username=handle)

followers = client.get_users_followers(
    # Learn about user fields here: 
    # https://developer.twitter.com/en/docs/twitter-api/data-dictionary/object-model/user
    user_obj.data.id, user_fields=["created_at","description","location",
                                   "public_metrics"]
)

Now let's explore these a bit. We'll start by printing out names, locations, following count, and followers count for these users.

In [8]:
num_to_print = 20

for idx, user in enumerate(followers.data) :
    following_count = user.public_metrics['following_count']
    followers_count = user.public_metrics['followers_count']
    
    print(f"{user.name} lists '{user.location}' as their location.")
    print(f" Following: {following_count}, Followers: {followers_count}.")
    print()
    
    if idx >= (num_to_print - 1) :
        break
    

Dave Renn lists 'None' as their location.
 Following: 42, Followers: 10.

Lionel lists 'None' as their location.
 Following: 202, Followers: 204.

Megan Randall lists 'None' as their location.
 Following: 141, Followers: 100.

Jacob Salzman lists 'None' as their location.
 Following: 562, Followers: 134.

twiter not fun lists 'None' as their location.
 Following: 220, Followers: 21.

Hariettwilsonincarnate lists 'None' as their location.
 Following: 218, Followers: 60.

Christian Tinsley lists 'None' as their location.
 Following: 2, Followers: 0.

Steve lists 'I'm over here.' as their location.
 Following: 1589, Followers: 33.

John O'Connor 🇺🇦 lists 'None' as their location.
 Following: 8, Followers: 1.

CodeGrade lists 'Amsterdam' as their location.
 Following: 2820, Followers: 424.

Cleverhood lists 'Providence, RI' as their location.
 Following: 2793, Followers: 3560.

Regina 🚶‍♀️🚲🌳 lists 'Minneapolis' as their location.
 Following: 2796, Followers: 3321.

Eric Hallstrom lists 'Mi

Let's find the person who follows this handle who has the most followers.


In [9]:
max_followers = 0

for idx, user in enumerate(followers.data) :
    followers_count = user.public_metrics['followers_count']
    
    if followers_count > max_followers :
        max_followers = followers_count
        max_follower_user = user

        
print(max_follower_user)
print(max_follower_user.public_metrics)

WedgeLIVE
{'followers_count': 14170, 'following_count': 2222, 'tweet_count': 56102, 'listed_count': 218}


Let's pull some more user fields and take a look at them. The fields can be specified in the user_fields argument.



In [10]:
response = client.get_user(id=user_obj.data.id,
                          user_fields=["created_at","description","location",
                                       "entities","name","pinned_tweet_id","profile_image_url",
                                       "verified","public_metrics"])

In [11]:
for field, value in response.data.items() :
    print(f"for {field} we have {value}")

for description we have He/Him. Data scientist, urban cyclist, educator, erstwhile frisbee player. 

¯\_(ツ)_/¯
for id we have 33029025
for verified we have False
for location we have MN
for profile_image_url we have https://pbs.twimg.com/profile_images/2680483898/b30ae76f909352dbae5e371fb1c27454_normal.png
for name we have John Chandler
for public_metrics we have {'followers_count': 193, 'following_count': 589, 'tweet_count': 996, 'listed_count': 3}
for created_at we have 2009-04-18 22:08:22+00:00
for username we have 37chandler


Now a few questions for you about the user object.

Q: How many fields are being returned in the response object?

A: 9


Q: Are any of the fields within the user object non-scalar? (I.e., more complicated than a simple data type like integer, float, string, boolean, etc.)

A: Public metrics have more than one data type.


Q: How many friends, followers, and tweets does this user have?

A: 589 friends, 193 followers, and 994 tweet counts. 

Although you won't need it for this assignment, individual tweets can be a rich source of text-based data. To illustrate the concepts, let's look at the last few tweets for this user. You are encouraged to explore the fields that are available about Tweets.

In [12]:
response = client.get_users_tweets(user_obj.data.id)

# By default, only the ID and text fields of each Tweet will be returned
for idx, tweet in enumerate(response.data) :
    print(tweet.id)
    print(tweet.text)
    print()
    
    if idx > 10 :
        break

1569155273742327811
As a Minneapolis person, I knew we had Toronto beat, but I didn't realize Portland had us beat: https://t.co/xrx5mOFcWK.

But @nytimes, c'mon! https://t.co/M9mBWhdgsj

1568982292923826176
RT @wonderofscience: Amazing lenticular cloud over Mount Fuji

Credit: Iurie Belegurschi
https://t.co/0mUxl28H9U

1568242374085869570
RT @depthsofwiki: lots of memes about speedy wikipedia editors — quick thread about what went down on wikipedia in the minutes after her de…

1568074978754703361
@DrLaurenWilson @leighradwood @MaritsaGeorgiou @Walgreens I could not possibly agree more with this sentiment. Compared to almost any other primary care I've received, they are great.

1567530169686196224
@DrLaurenWilson @MaritsaGeorgiou @Walgreens For those who have access to Curry Health Center on campus, you can get a bivalent booster in 15 minutes from their delightful staff.

1567511181526708224
RT @shes_the_maNN1: I can’t describe how ancient this makes me feel. https://t.co/a1IvELjOFY

###**Pulling Follower Information**


In this next section of the assignment, we will pull information about the followers of your two artists. We've seen above how to pull a set of followers using **client.get_users_followers**. This function has a parameter, max_results, that we can use to change the number of followers that we pull. Unfortunately, we can only pull 1000 followers at a time, which means we will need to handle the pagination of our results.

The return object has the .data field, where the results will be found. It also has .meta, which we use to select the next "page" in the results using the next_token result. I will illustrate the ideas using our user from above.

###Rate Limiting
Twitter limits the rates at which we can pull data, as detailed in this guide. We can make 15 user requests per 15 minutes, meaning that we can pull  users per hour. I illustrate the handling of rate limiting below, though whether or not you hit that part of the code depends on your value of handle.

In the below example, I'll pull all the followers, 25 at a time. (We're using 25 to illustrate the idea; when you do this set the value to 1000.)

TODO: Get artist counts and estimate total time.



Now let's take a look at your artists and see how long it is going to take to pull all their followers.



In [50]:
artists = dict()

for handle in ['echosmith','shayneTward'] : 
    user_obj = client.get_user(username=handle,user_fields=["public_metrics"])
    artists[handle] = (user_obj.data.id, 
                       handle,
                       user_obj.data.public_metrics['followers_count'])
    

for artist, data in artists.items() : 
    print(f"It would take {data[2]/(1000*15*4):.2f} hours to pull all {data[2]} followers for {artist}. ")
    

It would take 2.88 hours to pull all 172555 followers for echosmith. 
It would take 5.82 hours to pull all 349148 followers for shayneTward. 


Depending on what you see in the display above, you may want to limit how many followers you pull. It'd be great to get at least 200,000 per artist.

As we pull data for each artist we will write their data to a folder called "twitter", so we will make that folder if needed.


In [14]:

# Make the "twitter" folder here. If you'd like to practice your programming, add functionality 
# that checks to see if the folder exists. If it does, then "unlink" it. Then create a new one.

if os.path.isdir("twitter"):
    shutil.rmtree("twitter/")
os.mkdir("twitter")

In this following cells, build on the above code to pull some of the followers and their data for your two artists. As you pull the data, write the follower ids to a file called [artist name]_followers.txt in the "twitter" folder. For instance, for Cher I would create a file named cher_followers.txt. As you pull the data, also store it in an object like a list or a data frame.

Extract and store the following fields:

screen_name

name

id

location

followers_count

friends_count

description

Store the fields with one user per row in a tab-delimited text file with the name [artist name]_follower_data.txt. For instance, for Cher I would create a file named cher_follower_data.txt.



###**Scraping for Artist_1: Ecosmith**

In [15]:
# Modify the below code stub to pull the follower IDs and write them to a file. 

handles = ['ecosmith']
user_obj = client.get_user(username=handle)
handle_id = user_obj.data.id

whitespace_pattern = re.compile(r"\s+")

user_data = dict() 
followers_data = dict()

for handle in handles :
    user_data[handle] = [] # will be a list of lists
    followers_data[handle] = [] # will be a simple list of IDs


# Grabs the time when we start making requests to the API
start_time = datetime.datetime.now()

for handle in handles :
    
    # Create the output file names 
    
    followers_output_file = handle + "_followers.txt"
    user_data_output_file = handle + "_follower_data.txt"
    
    
    # Using tweepy.Paginator (https://docs.tweepy.org/en/latest/v2_pagination.html), 
    # use `get_users_followers` to pull the follower data requested. 
    
    for response in tweepy.Paginator(client.get_users_followers, handle_id,
                                 user_fields = ["id","username","name","location","public_metrics","description"],
                                    max_results=1000, limit=100):
      print(response.meta)
      
      for follower in response.data :
        follower_row = {
        'ID' : follower.id,
        'screen_name' : follower.username,
        'name' : follower.name,
        'location' : follower.location,
        'follower_count' : follower.public_metrics['followers_count'],
        'friends_count' : follower.public_metrics['following_count'],
        'description': re.sub(r"\s+"," ", follower.description).strip(),
        }
        
        user_data[handle].append(follower_row)   # All lists
        followers_data[handle].append(follower.id)  # just IDs
  
  #Save the extracted followers to a dataframe
    columns = ['ID', 'screen_name', 'name', 'location', 'follower_count', 'friends_count', 'description']
    columns_2 = ['ID']
    df = pd.DataFrame(user_data[handle], columns=columns)
    df2 = pd.DataFrame(followers_data[handle], columns = columns_2)
  
  
  #Write to the output file
    df.to_csv(f'twitter/{handle}_followers.txt', sep = '\t', index = False)
    df2.to_csv(f'twitter/{handle}_follower_data.txt', sep = '\t', index = False)
     

# Let's see how long it took to grab all follower IDs
end_time = datetime.datetime.now()
print(end_time - start_time)


{'result_count': 1000, 'next_token': 'BLE5JOD3C0D1GZZZ'}
{'result_count': 1000, 'next_token': 'FM62C5UMDG11GZZZ', 'previous_token': 'LSAUANN3JVIUEZZZ'}
{'result_count': 1000, 'next_token': 'BR2CUQTBJJNHEZZZ', 'previous_token': '2R24A7OVIJUUEZZZ'}
{'result_count': 1000, 'next_token': '40PAGGOQ0V7HEZZZ', 'previous_token': 'C8SL8R0OFS8EGZZZ'}
{'result_count': 1000, 'next_token': '92JO19E0DMQ1EZZZ', 'previous_token': '1P0JFT7JVCOEGZZZ'}
{'result_count': 1000, 'next_token': 'IGB44E0DPA1HEZZZ', 'previous_token': 'O2H05GFLK55UGZZZ'}
{'result_count': 1000, 'next_token': 'NVA79CKKJPD1EZZZ', 'previous_token': '7NJ53A3C75UEGZZZ'}
{'result_count': 1000, 'next_token': 'C5G6Q8S87KOHEZZZ', 'previous_token': 'VT3P7MSND6IUGZZZ'}
{'result_count': 1000, 'next_token': 'HAVMA4MSM0F1EZZZ', 'previous_token': 'NQDJKG4IOB7EGZZZ'}
{'result_count': 1000, 'next_token': 'PTKN0NASFO3HEZZZ', 'previous_token': '527GBHIKA7GUGZZZ'}
{'result_count': 1000, 'next_token': 'FD44T60C87KHCZZZ', 'previous_token': 'K9IR9R7EGNSE

{'result_count': 1000, 'next_token': 'GI0L95PVAUEHCZZZ', 'previous_token': 'JHM3JUFT0H4UIZZZ'}
{'result_count': 1000, 'next_token': '8KU4LHV0HI31CZZZ', 'previous_token': 'P8RLCHISM1HEIZZZ'}
{'result_count': 1000, 'next_token': 'P1SID1547TP1CZZZ', 'previous_token': '2OVB4LUHFHSUIZZZ'}
{'result_count': 1000, 'next_token': 'JJBUHF760DGHCZZZ', 'previous_token': 'TQ3CUASUOU6UIZZZ'}
{'result_count': 1000, 'next_token': 'FCDOOGP991CHCZZZ', 'previous_token': '2TOF2RSJVQFEIZZZ'}
{'result_count': 1000, 'next_token': 'O8VIQ8QL2D71CZZZ', 'previous_token': 'REDQVAO3N2JEIZZZ'}
{'result_count': 1000, 'next_token': 'AUD13J8PT50HCZZZ', 'previous_token': 'L9GT1GCUU2OUIZZZ'}
{'result_count': 1000, 'next_token': 'MEM76GGUOCTHCZZZ', 'previous_token': 'D9O74QP72UVEIZZZ'}
{'result_count': 1000, 'next_token': 'ICM8BSRK5ORHCZZZ', 'previous_token': 'IL82VQ4T7R2EIZZZ'}
{'result_count': 1000, 'next_token': 'SK2S96BP80QHCZZZ', 'previous_token': 'MNU3PRTMQ74EIZZZ'}
{'result_count': 1000, 'next_token': 'TVA2OI3RN8Q1

{'result_count': 1000, 'next_token': 'R1EUDPNNVCPHCZZZ', 'previous_token': 'EJQL44M0O75UIZZZ'}
{'result_count': 1000, 'next_token': 'UBQS6V6MP8PHCZZZ', 'previous_token': 'V9VBJBG90J6EIZZZ'}
{'result_count': 1000, 'next_token': 'CH5QL8GTOKPHCZZZ', 'previous_token': 'KB4FU7PF6N6EIZZZ'}
{'result_count': 1000, 'next_token': 'H8TGDJVFO8PHCZZZ', 'previous_token': '5AV6C8737B6EIZZZ'}
{'result_count': 1000, 'next_token': 'GEABEBCBMOOHCZZZ', 'previous_token': 'QQRQGFOG7N6EIZZZ'}
{'result_count': 1000, 'next_token': '6OPJ1JBSHKLHCZZZ', 'previous_token': 'VPHPVS3L977EIZZZ'}
{'result_count': 1000, 'next_token': 'PMTN6D177GJ1CZZZ', 'previous_token': '010JUFH7EJAEIZZZ'}
{'result_count': 1000, 'next_token': '6M34OO808CH1CZZZ', 'previous_token': 'NVRT1JVROFCUIZZZ'}
{'result_count': 1000, 'next_token': 'I4M8F43IIKEHCZZZ', 'previous_token': '0BBR3VCLNNEUIZZZ'}
{'result_count': 1000, 'next_token': '91HL5K8LBGC1CZZZ', 'previous_token': 'OOOI389SDFHEIZZZ'}
{'result_count': 1000, 'next_token': '3EPS02P18S91

{'result_count': 1000, 'next_token': '00GVHAS0HC11CZZZ', 'previous_token': 'QNUG5OUQCRTEIZZZ'}
{'result_count': 1000, 'next_token': 'LCKUJ9FCMBTHAZZZ', 'previous_token': '14JPOQENEJUUIZZZ'}
{'result_count': 1000, 'next_token': 'P34229QMUJQ1AZZZ', 'previous_token': 'JE77PUH89K2EKZZZ'}
{'result_count': 999, 'next_token': 'U4SJ7QKSRRNHAZZZ', 'previous_token': '3135B38J1G5UKZZZ'}
{'result_count': 1000, 'next_token': '5QQNFVDDVJLHAZZZ', 'previous_token': '432SN468488EKZZZ'}
{'result_count': 1000, 'next_token': 'JD5L0SQB6NL1AZZZ', 'previous_token': 'BA69N5TB0CAEKZZZ'}
{'result_count': 1000, 'next_token': '92PN14NPMVJ1AZZZ', 'previous_token': 'EDHKD632PCAUKZZZ'}
{'result_count': 1000, 'next_token': 'MFHA9AICCRH1AZZZ', 'previous_token': '3KR116T09GCUKZZZ'}
{'result_count': 1000, 'next_token': '5A3IL32L7BEHAZZZ', 'previous_token': 'JO9NN4N8J8EUKZZZ'}
{'result_count': 1000, 'next_token': 'TLA7238SR7BHAZZZ', 'previous_token': '1791JSDUP4HEKZZZ'}
{'result_count': 1000, 'next_token': 'TK42BDN9PV91A

{'result_count': 1000, 'next_token': '4UMSI4VB671HAZZZ', 'previous_token': 'N2CNQ5SSACRUKZZZ'}
{'result_count': 1000, 'next_token': 'I16AQ6O72IVHAZZZ', 'previous_token': 'FU7NOOBEQCUEKZZZ'}
{'result_count': 1000, 'next_token': '0NDJQTFIM6THAZZZ', 'previous_token': '806V2NGETH0EKZZZ'}
{'result_count': 1000, 'next_token': 'GOROAGJOPASHAZZZ', 'previous_token': 'TANPNGI69P2EKZZZ'}
{'result_count': 1000, 'next_token': 'TV7BL9S9EIS1AZZZ', 'previous_token': '3NE2HUO36P3EKZZZ'}
{'result_count': 1000, 'next_token': 'M8AHGH4N86R1AZZZ', 'previous_token': 'F8IG80C1HD3UKZZZ'}
{'result_count': 1000, 'next_token': '7GS55QQ1JMQ1AZZZ', 'previous_token': 'CMJIN27RNT4UKZZZ'}
{'result_count': 1000, 'next_token': '0QJTTK8P9IPHAZZZ', 'previous_token': '7OBHEGN5C95UKZZZ'}
{'result_count': 1000, 'next_token': 'BEQLRHGM9UNHAZZZ', 'previous_token': 'G8UD81VNMD6EKZZZ'}
{'result_count': 1000, 'next_token': 'QTC8UH4IBQMHAZZZ', 'previous_token': 'IDM7V4OPM98EKZZZ'}
{'result_count': 1000, 'next_token': '8CQB138N8AM1

{'result_count': 1000, 'next_token': 'HJCIL88LV6GHAZZZ', 'previous_token': 'AE0J3VJ5RDCEKZZZ'}
{'result_count': 1000, 'next_token': 'NS7K6VQUFUEHAZZZ', 'previous_token': 'D3BKSAJU0TFEKZZZ'}
{'result_count': 1000, 'next_token': 'F175P79MSED1AZZZ', 'previous_token': '89PDPGEFG1HEKZZZ'}
{'result_count': 1000, 'next_token': 'VENURR1JSQC1AZZZ', 'previous_token': 'ROR06OUC3LIUKZZZ'}
{'result_count': 1000, 'next_token': '7DO5TQ9M8MBHAZZZ', 'previous_token': 'A2OB1O2T39JUKZZZ'}
{'result_count': 1000, 'next_token': 'JPTJF5TOUUAHAZZZ', 'previous_token': 'U9KKTFFGN9KEKZZZ'}
{'result_count': 1000, 'next_token': '5AML1FQBCEA1AZZZ', 'previous_token': '91TEHTJH11LEKZZZ'}
{'result_count': 1000, 'next_token': 'DL8BUPMULU91AZZZ', 'previous_token': '02V20RUTJHLUKZZZ'}
{'result_count': 1000, 'next_token': 'HSE4VQ913U81AZZZ', 'previous_token': 'O3SJOPMIA1MUKZZZ'}
{'result_count': 1000, 'next_token': 'HECQUNPSJU61AZZZ', 'previous_token': '69S6NCJQS5NUKZZZ'}
{'result_count': 1000, 'next_token': '9VHRVA78FU41

{'result_count': 1000, 'next_token': '737JJRCKHTUHAZZZ', 'previous_token': 'UL55RR8ADE0UKZZZ'}
{'result_count': 1000, 'next_token': 'FS3PUFQ8A5U1AZZZ', 'previous_token': 'P17NH4ROE21EKZZZ'}
{'result_count': 999, 'next_token': 'B6FQGKAQT9T1AZZZ', 'previous_token': '4M69M6V2LQ1UKZZZ'}
{'result_count': 1000, 'next_token': '02P51KA7H1SHAZZZ', 'previous_token': '74QKJV052U2UKZZZ'}
{'result_count': 1000, 'next_token': '8O58I3URC5S1AZZZ', 'previous_token': '7NSMV0RBF23EKZZZ'}
{'result_count': 1000, 'next_token': 'GEB597K65TRHAZZZ', 'previous_token': 'I2SUKB9AJQ3UKZZZ'}
{'result_count': 1000, 'next_token': '4MBTVGURK1QHAZZZ', 'previous_token': 'JE1R8K3QQ24EKZZZ'}
{'result_count': 1000, 'next_token': 'SIVHHL7FPHPHAZZZ', 'previous_token': 'U1BG7PKBC65EKZZZ'}
{'result_count': 1000, 'next_token': 'LRVORESVVTOHAZZZ', 'previous_token': '4FBK7NIH6E6EKZZZ'}
{'result_count': 1000, 'next_token': 'I18RTD9PHTO1AZZZ', 'previous_token': '74MKMR3L027EKZZZ'}
{'result_count': 1000, 'next_token': 'C2EQS60I19NHA

###**Scraping for Artist_2: ShayneWard**

In [62]:
# Modify the below code stub to pull the follower IDs and write them to a file. 

handles = ['shayneTward']
user_obj = client.get_user(username=handle)
handle_id = user_obj.data.id

whitespace_pattern = re.compile(r"\s+")

user_data = dict() 
followers_data = dict()

for handle in handles :
    user_data[handle] = [] # will be a list of lists
    followers_data[handle] = [] # will be a simple list of IDs


# Grabs the time when we start making requests to the API
start_time = datetime.datetime.now()

for handle in handles :
    
    # Create the output file names 
    
    followers_output_file = handle + "_followers.txt"
    user_data_output_file = handle + "_follower_data.txt"
    
    
    # Using tweepy.Paginator (https://docs.tweepy.org/en/latest/v2_pagination.html), 
    # use `get_users_followers` to pull the follower data requested. 
    
    for response in tweepy.Paginator(client.get_users_followers, handle_id,
                                 user_fields = ["id","username","name","location","public_metrics","description"],
                                    max_results=1000, limit=100):
      print(response.meta)
      
      for follower in response.data :
        follower_row = {
        'ID' : follower.id,
        'screen_name' : follower.username,
        'name' : follower.name,
        'location' : follower.location,
        'follower_count' : follower.public_metrics['followers_count'],
        'friends_count' : follower.public_metrics['following_count'],
        'description': re.sub(r"\s+"," ", follower.description).strip(),
        }
        
        user_data[handle].append(follower_row)   # All lists
        followers_data[handle].append(follower.id)  # just IDs
  
  #Save the extracted followers to a dataframe
    columns = ['ID', 'screen_name', 'name', 'location', 'follower_count', 'friends_count', 'description']
    columns_3 = ['ID']
    df3 = pd.DataFrame(user_data[handle], columns=columns)
    df4 = pd.DataFrame(followers_data[handle], columns = columns_3)
  
  
  #Write to the output file
    df3.to_csv(f'twitter/{handle}_followers.txt', sep = '\t', index = False)
    df4.to_csv(f'twitter/{handle}_follower_data.txt', sep = '\t', index = False)
     

# Let's see how long it took to grab all follower IDs
end_time = datetime.datetime.now()
print(end_time - start_time)


{'result_count': 1000, 'next_token': 'RLHD2NQ8CCD1GZZZ'}
{'result_count': 1000, 'next_token': 'SRGHEO3VE411GZZZ', 'previous_token': 'KEHQC7ISJVIUEZZZ'}
{'result_count': 1000, 'next_token': 'ET18NQ0QKFNHEZZZ', 'previous_token': 'GDPTJQ19IFUUEZZZ'}
{'result_count': 1000, 'next_token': '3G13DRU50V7HEZZZ', 'previous_token': 'K8TJ152KCC8EGZZZ'}
{'result_count': 1000, 'next_token': 'GJCDQGO4JQQ1EZZZ', 'previous_token': 'S36LFF75V0OEGZZZ'}
{'result_count': 1000, 'next_token': 'C21H51UDTI1HEZZZ', 'previous_token': 'N1C7UMHVI95UGZZZ'}
{'result_count': 1000, 'next_token': 'PO6Q1M4LJPD1EZZZ', 'previous_token': 'DJKRRHVI6LUEGZZZ'}
{'result_count': 1000, 'next_token': '7CVHRS927OOHEZZZ', 'previous_token': '84LOMJBBC6IUGZZZ'}
{'result_count': 1000, 'next_token': '0F0VKD4EMGF1EZZZ', 'previous_token': 'JUFP5N3NOB7EGZZZ'}
{'result_count': 1000, 'next_token': '5RNTVHH0H03HEZZZ', 'previous_token': 'EP09LR939VGUGZZZ'}
{'result_count': 1000, 'next_token': '5A65J68T9NKHCZZZ', 'previous_token': '66B8V8L3G7SE

{'result_count': 1000, 'next_token': 'P3QEPCE5HM31CZZZ', 'previous_token': 'FHVAMQ60L1HEIZZZ'}
{'result_count': 1000, 'next_token': 'E9IJOG9S8LP1CZZZ', 'previous_token': 'NF1RAE0VEDSUIZZZ'}
{'result_count': 1000, 'next_token': 'M9EKKLEG0LGHCZZZ', 'previous_token': '723DIUQRO26UIZZZ'}
{'result_count': 1000, 'next_token': '782P042S91CHCZZZ', 'previous_token': 'CGK1EGOPVIFEIZZZ'}
{'result_count': 1000, 'next_token': '9NUMMC442H71CZZZ', 'previous_token': 'GNI77F6MMUJEIZZZ'}
{'result_count': 1000, 'next_token': 'I6BOF7HAT50HCZZZ', 'previous_token': '7R0D5N5ATIOUIZZZ'}
{'result_count': 1000, 'next_token': 'VLAUC7NFOGTHCZZZ', 'previous_token': 'L5IUSCN62QVEIZZZ'}
{'result_count': 1000, 'next_token': '5AC6LF6E5SRHCZZZ', 'previous_token': 'VE13D4UG7F2EIZZZ'}
{'result_count': 1000, 'next_token': '3JQNJLTD84QHCZZZ', 'previous_token': 'JCTKDBJMQ34EIZZZ'}
{'result_count': 1000, 'next_token': 'T7117PKUN8Q1CZZZ', 'previous_token': 'IMADCU2PNV5EIZZZ'}
{'result_count': 1000, 'next_token': '1Q8S9SHDHKQ1

{'result_count': 1000, 'next_token': 'EQNCFPUSP8PHCZZZ', 'previous_token': 'N6RDSAG30J6EIZZZ'}
{'result_count': 1000, 'next_token': 'FNPMP98UOKPHCZZZ', 'previous_token': 'NOPCC9P76N6EIZZZ'}
{'result_count': 1000, 'next_token': '3RQ1KI7GO8PHCZZZ', 'previous_token': 'UE82IU717B6EIZZZ'}
{'result_count': 1000, 'next_token': 'HUSBD7ENMOOHCZZZ', 'previous_token': 'BEL8O0OG7N6EIZZZ'}
{'result_count': 1000, 'next_token': 'NKPVAQQAI0LHCZZZ', 'previous_token': '7LAJCQA6977EIZZZ'}
{'result_count': 1000, 'next_token': 'PP7T5BEU7GJ1CZZZ', 'previous_token': '9RHRP0QMEBAEIZZZ'}
{'result_count': 1000, 'next_token': 'K1NEQTV58GH1CZZZ', 'previous_token': 'IUNPMMS1OFCUIZZZ'}
{'result_count': 1000, 'next_token': 'OHEF5DQTIOEHCZZZ', 'previous_token': 'R05HCS9LNFEUIZZZ'}
{'result_count': 1000, 'next_token': 'ASN8NIKDBGC1CZZZ', 'previous_token': 'NF8IMBM1D7HEIZZZ'}
{'result_count': 1000, 'next_token': 'BOASP0J18S91CZZZ', 'previous_token': 'V9J2TV4KKFJUIZZZ'}
{'result_count': 1000, 'next_token': '1EL3EA24K86H

{'result_count': 1000, 'next_token': 'ET7UVL5PMNTHAZZZ', 'previous_token': 'IB641537EFUUIZZZ'}
{'result_count': 1000, 'next_token': 'EQGS6EKRURQ1AZZZ', 'previous_token': 'S9HFLDP59G2EKZZZ'}
{'result_count': 999, 'next_token': '2VCJAKUGRRNHAZZZ', 'previous_token': 'DEPODH121C5UKZZZ'}
{'result_count': 1000, 'next_token': 'HQMESPF2VJLHAZZZ', 'previous_token': 'B5I9KKHP448EKZZZ'}
{'result_count': 1000, 'next_token': 'OKRAR89I6RL1AZZZ', 'previous_token': 'SHM4D19U0CAEKZZZ'}
{'result_count': 1000, 'next_token': '9UUP96E3NFJ1AZZZ', 'previous_token': '9SBRQMQRP8AUKZZZ'}
{'result_count': 1000, 'next_token': 'CN2EQEFPE7H1AZZZ', 'previous_token': 'OJ4QP7M88OCUKZZZ'}
{'result_count': 1000, 'next_token': 'NNEORLV3AREHAZZZ', 'previous_token': '6C3FKSLAHOEUKZZZ'}
{'result_count': 1000, 'next_token': 'IDJPKVSJRFBHAZZZ', 'previous_token': '2SCUMUJPM4HEKZZZ'}
{'result_count': 1000, 'next_token': '6KGRPIH5Q791AZZZ', 'previous_token': 'U2BO5QIB4KKEKZZZ'}
{'result_count': 1000, 'next_token': 'CN169PMPBR71A

{'result_count': 1000, 'next_token': 'RN9P302E2MVHAZZZ', 'previous_token': 'SHN07IMQPKUEKZZZ'}
{'result_count': 1000, 'next_token': 'A7I1FU6SMATHAZZZ', 'previous_token': 'OB6V310CTD0EKZZZ'}
{'result_count': 1000, 'next_token': '956T9QS1PESHAZZZ', 'previous_token': 'I4E0II8A9P2EKZZZ'}
{'result_count': 1000, 'next_token': 'TG6AHT4HEIS1AZZZ', 'previous_token': '7R7LVPTH6H3EKZZZ'}
{'result_count': 1000, 'next_token': 'JLJGQJFJ86R1AZZZ', 'previous_token': 'STCLV63HHD3UKZZZ'}
{'result_count': 1000, 'next_token': 'UCQKK7FLJMQ1AZZZ', 'previous_token': '9RLEFER8NP4UKZZZ'}
{'result_count': 1000, 'next_token': 'VNAGU2HH9IPHAZZZ', 'previous_token': 'OJ3QQ55UC95UKZZZ'}
{'result_count': 1000, 'next_token': '53AU73SR9UNHAZZZ', 'previous_token': 'V9C22BN6MD6EKZZZ'}
{'result_count': 1000, 'next_token': 'CCMP9KCUBUMHAZZZ', 'previous_token': 'KL5A4EF9M18EKZZZ'}
{'result_count': 1000, 'next_token': 'MSIR6GKS8AM1AZZZ', 'previous_token': '56JN1ERDK59EKZZZ'}
{'result_count': 1000, 'next_token': 'QMOS77JMK2L1

{'result_count': 1000, 'next_token': '67CTK8JMFUEHAZZZ', 'previous_token': 'EGJDANNA0PFEKZZZ'}
{'result_count': 1000, 'next_token': 'D00JPS2ASED1AZZZ', 'previous_token': '87OBP051G1HEKZZZ'}
{'result_count': 1000, 'next_token': '8TB8UDTDSQC1AZZZ', 'previous_token': 'H2OQ6OM93HIUKZZZ'}
{'result_count': 1000, 'next_token': 'K0LIDERR8MBHAZZZ', 'previous_token': '0L8144UC35JUKZZZ'}
{'result_count': 1000, 'next_token': 'CCQUN20RV2AHAZZZ', 'previous_token': 'OM7Q25M9N9KEKZZZ'}
{'result_count': 1000, 'next_token': 'Q8CSADKVCEA1AZZZ', 'previous_token': 'CA2CGQ2711LEKZZZ'}
{'result_count': 1000, 'next_token': '1K1F0RCUM691AZZZ', 'previous_token': 'QP9AUG5KJHLUKZZZ'}
{'result_count': 1000, 'next_token': 'F94B7E9R3U81AZZZ', 'previous_token': 'IENK1691A1MUKZZZ'}
{'result_count': 1000, 'next_token': 'VBLSD043JU61AZZZ', 'previous_token': 'E7HR05MUS1NUKZZZ'}
{'result_count': 1000, 'next_token': 'ASL7DF9TG641AZZZ', 'previous_token': 'ELJ51863C1PUKZZZ'}
{'result_count': 1000, 'next_token': 'I9GCJ6F2TM21

{'result_count': 1000, 'next_token': '37E6T2R2A5U1AZZZ', 'previous_token': 'P0OCC4JBE21EKZZZ'}
{'result_count': 999, 'next_token': 'GGVDLORMT9T1AZZZ', 'previous_token': 'G7S61G5NLQ1UKZZZ'}
{'result_count': 1000, 'next_token': 'SBMM57BLH5SHAZZZ', 'previous_token': 'KTG5FBL52M2UKZZZ'}
{'result_count': 1000, 'next_token': 'E9KI0AV3C5S1AZZZ', 'previous_token': '017AUBLOEU3EKZZZ'}
{'result_count': 1000, 'next_token': 'MB68M2LF5TRHAZZZ', 'previous_token': 'NBQNDS14JQ3UKZZZ'}
{'result_count': 1000, 'next_token': 'Q5KB0FIVKHQHAZZZ', 'previous_token': 'FLKQMOBPQ24EKZZZ'}
{'result_count': 1000, 'next_token': 'CPD6IPNRPHPHAZZZ', 'previous_token': 'RDK20F14BU5EKZZZ'}
{'result_count': 1000, 'next_token': '5BLQPF6AVTOHAZZZ', 'previous_token': '3H0EEAOG6E6EKZZZ'}
{'result_count': 1000, 'next_token': '57BUR635HTO1AZZZ', 'previous_token': 'A8074H30027EKZZZ'}
{'result_count': 1000, 'next_token': 'SQ0P3V2Q19NHAZZZ', 'previous_token': 'E2N42IM6E27UKZZZ'}
{'result_count': 1000, 'next_token': '9AKS6J1QRHM1A

In [63]:
tricky_description = """
    Home by Warsan Shire
    
    no one leaves home unless
    home is the mouth of a shark.
    you only run for the border
    when you see the whole city
    running as well.

"""
# This won't work in a tab-delimited text file.

clean_description = re.sub(r"\s+"," ",tricky_description).strip()
clean_description

'Home by Warsan Shire no one leaves home unless home is the mouth of a shark. you only run for the border when you see the whole city running as well.'

##**Lyrics Scraping**

This section asks you to pull data from the Twitter API and scrape www.AZLyrics.com. In the notebooks where you do that work you are asked to store the data in specific ways.

In [64]:
artists = {'EcoSmith':"https://www.azlyrics.com/e/echosmith.html",
    'ShayneWard':"https://www.azlyrics.com/s/shayneward.html"} 
# we'll use this dictionary to hold both the artist name and the link on AZlyrics

###**A Note on Rate Limiting**

The lyrics site, www.azlyrics.com, does not have an explicit maximum on number of requests in any one time, but in our testing it appears that too many requests in too short a time will cause the site to stop returning lyrics pages. (Entertainingly, the page that gets returned seems to only have the song title to a Tom Jones song.)

Whenever you call requests.get to retrieve a page, put a time.sleep(5 + 10*random.random()) on the next line. This will help you not to get blocked. If you do get blocked, which you can identify if the returned pages are not correct, just request a lyrics page through your browser. You'll be asked to perform a CAPTCHA and then your requests should start working again.




###**Part 1: Finding Links to Songs Lyrics**

That general artist page has a list of all songs for that artist with links to the individual song pages.

Q: Take a look at the robots.txt page on www.azlyrics.com. (You can read more about these pages here.) Is the scraping we are about to do allowed or disallowed by this page? How do you know?

**A:  Scraping is allowed with Robots.txt.  But anything that include the /lyricsdb/ and the/song/paths are disallowed.** 



In [34]:
import random
# Let's set up a dictionary of lists to hold our links
lyrics_pages = defaultdict(list)

for artist, artist_page in artists.items() :
    # request the page and sleep
    r = requests.get(artist_page)
    time.sleep(5 + 10*random.random())

    # now extract the links to lyrics pages from this page
    soup = BeautifulSoup(r.content)
    links = soup.select('a[href^="/lyrics/"]')
    # store the links `lyrics_pages` where the key is the artist and the
    # value is a list of links. 
    lyrics_pages[artist] = ["https://www.azlyrics.com" + i.get("href") for i in links]

Let's make sure we have enough lyrics pages to scrape.

In [35]:
# Let's see how long it's going to take to pull these lyrics 
# if we're waiting `5 + 10*random.random()` seconds 
for artist, links in lyrics_pages.items() : 
    print(f"For {artist} we have {len(links)}.")
    print(f"The full pull will take for this artist will take {round(len(links)*10/3600,2)} hours.")

For EcoSmith we have 42.
The full pull will take for this artist will take 0.12 hours.
For ShayneWard we have 59.
The full pull will take for this artist will take 0.16 hours.


###**Part 2: Pulling Lyrics**

Now that we have the links to our lyrics pages, let's go scrape them! Here are the steps for this part.

Create an empty folder in our repo called "lyrics".

Iterate over the artists in lyrics_pages.

Create a subfolder in lyrics with the artist's name. For instance, if the artist was Cher you'd have lyrics/cher/ in your repo.

Iterate over the pages.

Request the page and extract the lyrics from the returned HTML file using BeautifulSoup.

Use the function below, generate_filename_from_url, to create a filename based on the lyrics page, then write the lyrics to a text file with that name.

In [36]:
def generate_filename_from_link(link) :
    
    if not link :
        return None
    
    # drop the http or https and the html
    name = link.replace("https","").replace("http","")
    name = link.replace(".html","")

    name = name.replace("/lyrics/","")
    
    # Replace useless chareacters with UNDERSCORE
    name = name.replace("://","").replace(".","_").replace("/","_")
    
    # tack on .txt
    name = name + ".txt"
    
    return(name)

In [37]:
# Make the lyrics folder here. If you'd like to practice your programming, add functionality 
# that checks to see if the folder exists. If it does, then use shutil.rmtree to remove it and create a new one.

if os.path.isdir("lyrics") : 
    shutil.rmtree("lyrics/")

os.mkdir("lyrics")

In [38]:
url_stub = "https://www.azlyrics.com" 
start = time.time()

total_pages = 0 

for artist in lyrics_pages :
    # Use this space to carry out the following steps: 
    
    # 1. Build a subfolder for the artist
    if os.path.isdir("lyrics/" + artist):
        shutil.rmtree("lyrics/" + artist)
    os.mkdir("lyrics/" + artist)
    path = "lyrics/" + artist

    # 2. Iterate over the lyrics pages
    # 3. Request the lyrics page.
     # Don't forget to add a line like `time.sleep(5 + 10*random.random()) to sleep after making the request 

    for song in lyrics_pages[artist]:
       
        r = requests.get(song)
        time.sleep(5 + 10*random.random())
        lyrics_soup = BeautifulSoup(r.content)

    # 4. Extract the title and lyrics from the page.
        initial = lyrics_soup.find("b")
        title = initial.find_next("b").get_text()
        lyrics = lyrics_soup.find("div", class_=False, id=False).get_text()

    # 5. Write out the title, two returns ('\n'), and the lyrics. Use `generate_filename_from_url to generate the filename. 
        lyrics_title = title + '\n' + '\n' + lyrics
        open_path = os.path.join(path, generate_filename_from_link(song))
        file = open(open_path,'w')
        file.write(lyrics_title)

    # Remember to pull at least 20 songs per artist. It may be fun to pull all the songs for the artist


In [39]:
print(f"Total run time was {round((time.time() - start)/3600,2)} hours.")

Total run time was 0.3 hours.


###**Evaluation**

This assignment asks you to pull data from the Twitter API and scrape www.AZLyrics.com. After you have finished the above sections , run all the cells in this notebook. Print this to PDF and submit it, per the instructions.

In [40]:
# Simple word extractor from Peter Norvig: https://norvig.com/spell-correct.html
def words(text): 
    return re.findall(r'\w+', text.lower())

###**Checking Twitter Data**


The output from your Twitter API pull should be two files per artist, stored in files with formats like cher_followers.txt (a list of all follower IDs you pulled) and cher_followers_data.txt. These files should be in a folder named twitter within the repository directory. This code summarizes the information at a high level to help the instructor evaluate your work.

In [91]:
twitter_files = os.listdir("twitter")
twitter_files = [f for f in twitter_files if f != ".DS_Store"]
artist_handles = ['ecosmith', 'shayneTward']
artist_handles
#artist_handles = list(set([name.split("_")[0] for name in twitter_files]))

print(f"We see two artist handles: {artist_handles[0]} and {artist_handles[1]}.")

We see two artist handles: ecosmith and shayneTward.


In [92]:
artist_handles

['ecosmith', 'shayneTward']

In [107]:
for artist in artist_handles :
    follower_file = artist + "_followers.txt"
    follower_data_file = artist + "_followers_data.txt"
    
    ids = open("twitter/" + follower_file,'r').readlines()
    
    print(f"We see {len(ids)-1} in your follower file for {artist}, assuming a header row.")
    
    with open("twitter/" + follower_file,'r') as infile :
        
        # check the headers
        headers = infile.readline().split("\t")
        
        print(f"In the follower data file ({follower_data_file}) for {artist}, we have these columns:")
        print(" : ".join(headers))
        
        description_words = []
        locations = set()
        
        
        for idx, line in enumerate(infile.readlines()) :
            line = line.strip("\n").split("\t")
            
            try : 
                locations.add(line[3])            
                description_words.extend(words(line[6]))
            except :
                pass
    
     
        print(f"We have {idx+1} data rows for {artist} in the follower data file.")

        print(f"For {artist} we have {len(locations)} unique locations.")

        print(f"For {artist} we have {len(description_words)} words in the descriptions.")
        print("Here are the five most common words:")
        print(Counter(description_words).most_common(5))

        
        print("")
        print("-"*40)
        print("")
    

We see 100024 in your follower file for ecosmith, assuming a header row.
In the follower data file (ecosmith_followers_data.txt) for ecosmith, we have these columns:
ID : screen_name : name : location : follower_count : friends_count : description

We have 100024 data rows for ecosmith in the follower data file.
For ecosmith we have 13001 unique locations.
For ecosmith we have 437174 words in the descriptions.
Here are the five most common words:
[('and', 13622), ('i', 10207), ('to', 8188), ('a', 8177), ('the', 7434)]

----------------------------------------

We see 100024 in your follower file for shayneTward, assuming a header row.
In the follower data file (shayneTward_followers_data.txt) for shayneTward, we have these columns:
ID : screen_name : name : location : follower_count : friends_count : description

We have 100024 data rows for shayneTward in the follower data file.
For shayneTward we have 13005 unique locations.
For shayneTward we have 437147 words in the descriptions.
H

###**Checking Lyrics**

The output from your lyrics scrape should be stored in files located in this path from the directory: /lyrics/[Artist Name]/[filename from URL]. 

In [87]:
artist_folders = os.listdir("lyrics/")
artist_folders = [f for f in artist_folders if os.path.isdir("lyrics/" + f)]

for artist in artist_folders : 
    artist_files = os.listdir("lyrics/" + artist)
    artist_files = [f for f in artist_files if 'txt' in f or 'csv' in f or 'tsv' in f]

    print(f"For {artist} we have {len(artist_files)} files.")

    artist_words = []

    for f_name in artist_files : 
        with open("lyrics/" + artist + "/" + f_name) as infile : 
            artist_words.extend(words(infile.read()))

            
    print(f"For {artist} we have roughly {len(artist_words)} words, {len(set(artist_words))} are unique.")

For ShayneWard we have 58 files.
For ShayneWard we have roughly 17812 words, 1326 are unique.
For EcoSmith we have 42 files.
For EcoSmith we have roughly 11175 words, 1040 are unique.
